 # 1. Setup Watson Machine Learning Connection

In [ ]:
PROJECT_ID = 'find me in the Watson Studio UI'
API_KEY = 'Create me in the IBM Cloud UI'
LOCATION = 'https://us-south.ml.cloud.ibm.com'

In [ ]:
# The code was removed by Watson Studio for sharing.

In [ ]:
from ibm_watson_machine_learning import APIClient

wml_credentials = {
    "apikey": API_KEY,
    "url": LOCATION
}

wml_client = APIClient(wml_credentials)

In [ ]:
wml_client.set.default_space('find me in Watson Machine Learning UI') 
wml_client.set.default_project(PROJECT_ID)

# 2. Setting up DB2 Connection

In [ ]:
import pandas as pd
import tensorflow as tf
import keras

In [ ]:
# 1. create a DB2 instance + upload your data
# 2. Create a "connection" in Watson Studio through the UI
# 3. Use the "insert to code" feature in this notebook

In [ ]:
# The code was removed by Watson Studio for sharing.

# 3. Prepare the data

In [ ]:
# convert all float columns to int
def convert_float_to_int(df):
    for col in df.columns:
        if df[col].dtype == 'float64':
            df[col] = df[col].astype('int64')
    return df

df=convert_float_to_int(df) 

In [ ]:
# ignoring all non numeric features for now (just to keep it simple)
numeric_feature_names = ['CUSTOMER_AGE', 'DEPENDENT_COUNT', 'MONTHS_ON_BOOK',  'TOTAL_RELATIONSHIP_COUNT', 'MONTHS_INACTIVE_12_MON','CONTACTS_COUNT_12_MON','CREDIT_LIMIT','TOTAL_REVOLVING_BAL','AVG_OPEN_TO_BUY','TOTAL_AMT_CHNG_Q4_Q1','TOTAL_TRANS_CT','TOTAL_CT_CHNG_Q4_Q1','AVG_UTILIZATION_RATIO']
numeric_features = df[numeric_feature_names]
numeric_features.head()

In [ ]:
train_dataset = numeric_features.sample(frac=0.8, random_state=0)
test_dataset = numeric_features.drop(train_dataset.index)

train_target = train_dataset.pop('CREDIT_LIMIT')
test_target = test_dataset.pop('CREDIT_LIMIT')

# 4. Do the actual Machine Learning

 https://www.tensorflow.org/tutorials/keras/regression

In [ ]:
# not using a normalizer since it causes issues when saving the model https://github.com/keras-team/keras/issues/15348
def get_basic_model():
  model = tf.keras.Sequential([
    #normalizer,
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(1)
  ])

  model.compile(optimizer='adam',
                loss=tf.keras.losses.MeanSquaredError(),
                metrics=['accuracy'])
  return model

In [ ]:
model = get_basic_model()
model.fit(train_dataset, train_target, epochs=100, batch_size=50, validation_split = 0.2)

In [ ]:
test_dataset.iloc[0]

In [ ]:
import numpy as np
# get random number
random_number = np.random.randint(0, len(test_dataset))
print("model result: ", model.predict(test_dataset.iloc[[random_number]])[0][0])
print("ground truth: ", test_target.iloc[[random_number]].values[0])

# 5. Storing the model

tar the model https://stackoverflow.com/questions/53258236/how-to-save-keras-model-in-wml-repository-in-watson-studio

Do this: https://github.com/IBM/watson-machine-learning-samples/blob/master/cpd3.5/notebooks/python_sdk/deployments/keras/Use%20Keras%20to%20recognize%20hand-written%20digits.ipynb

In [ ]:
model_result_path = "keras_model.h5"
model.save(model_result_path)

In [ ]:
!tar -zcvf keras_model.tgz keras_model.h5

In [ ]:
! ls -la

## 5.1. upload the model

In [ ]:
# get the tensorflow version so you can set it in the meta data
! pip list | grep tensorflow # get the tensorflow version for the model metadata 

In [ ]:
sofware_spec_uid = wml_client.software_specifications.get_id_by_name("runtime-22.2-py3.10")

metadata = {
            wml_client.repository.ModelMetaNames.NAME: 'credit limit model',
            wml_client.repository.ModelMetaNames.TYPE: 'tensorflow_2.9',
            wml_client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: sofware_spec_uid
}

published_model = wml_client.repository.store_model(
    model="keras_model.tgz",
    meta_props=metadata)

## 5.2. get the model information programatically

In [ ]:
import json

published_model_uid = wml_client.repository.get_model_id(published_model)
model_details = wml_client.repository.get_details(published_model_uid)
print(json.dumps(model_details, indent=2))

In [ ]:
models_details = wml_client.repository.list_models()

In [ ]:
metadata = {
    wml_client.deployments.ConfigurationMetaNames.NAME: "Deployment of external Keras model",
    wml_client.deployments.ConfigurationMetaNames.ONLINE: {}
}

created_deployment = wml_client.deployments.create(published_model_uid, meta_props=metadata)

## 5.3 test the deployed model

In [ ]:
deployment_uid = wml_client.deployments.get_id(created_deployment)
scoring_endpoint = wml_client.deployments.get_scoring_href(created_deployment)
print(scoring_endpoint)

In [ ]:
test_dataset.iloc[[2]]

In [ ]:
scoring_payload = {"input_data": [{"values": test_dataset.iloc[[2]]}]}
predictions = wml_client.deployments.score(deployment_uid, scoring_payload)
 
print("model result: ", predictions['predictions'][0]['values'][0][0][0])
print("ground truth: ", test_target.iloc[[2]].values[0])